好的,我們可以將梯度下降的概念延伸到反向傳播神經網絡(BPN)。以下是BPN的數學表示和簡單的Python實現:

1. 數學表示:

假設我們有一個簡單的三層神經網絡(輸入層、隱藏層、輸出層):

- 輸入: x = [x₁, x₂, ..., xₙ]
- 隱藏層權重: W₁, 偏置: b₁
- 輸出層權重: W₂, 偏置: b₂
- 激活函數: σ (例如sigmoid函數)

## 前向傳播:
## z₁ = W₁x + b₁
## a₁ = σ(z₁)
## z₂ = W₂a₁ + b₂
## y = σ(z₂)

## 反向傳播:
## δ₂ = (y - t) ⊙ σ'(z₂)  (t是目標輸出)
其中:
- δ₂: 輸出層的誤差
- y: 實際輸出
- t: 目標輸出
- ⊙: 哈達瑪積（元素間逐點相乘）
- σ': sigmoid函數的導數
- z₂: 輸出層的加權和

## δ₁ = (W₂ᵀδ₂) ⊙ σ'(z₁)

其中:
- δ₁: 隱藏層的誤差
- W₂ᵀ: W₂的轉置矩陣
- z₁: 隱藏層的加權和

## 更新規則:
## W₂ = W₂ - α * δ₂a₁ᵀ
## b₂ = b₂ - α * δ₂
## W₁ = W₁ - α * δ₁xᵀ
## b₁ = b₁ - α * δ₁


## 2. Python實現 BPN




In [4]:
import numpy as np  # 導入 NumPy 庫，用於進行數值計算和數組操作

# 定義 sigmoid 激活函數
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # 計算並返回 sigmoid 函數值

# 定義 sigmoid 函數的導數
def sigmoid_derivative(x):
    return x * (1 - x)  # 計算並返回 sigmoid 函數的導數

# 定義神經網絡類
class NeuralNetwork:
    def __init__(self, x, y):
        self.input = x  # 設置輸入數據
        self.weights1 = np.random.rand(self.input.shape[1], 4)  # 初始化第一層權重：使用numpy的random.rand函數生成一個隨機矩陣，大小為輸入特徵數量乘以4（隱藏層神經元數量）
        self.weights2 = np.random.rand(4, 1)  # 初始化第二層權重：生成一個4行1列的隨機矩陣，連接隱藏層到輸出層
        self.y = y  # 設置目標輸出：將傳入的目標值y賦給self.y，用於後續的訓練和誤差計算
        self.output = np.zeros(y.shape)  # 初始化輸出為零矩陣：創建一個與目標輸出y相同形狀的零矩陣，用於存儲網絡的預測結果

    # 前向傳播
    def feedforward(self):
        # 前向傳播函數
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))  # 計算隱藏層的輸出
        # 詳細解釋：
        # 1. np.dot(self.input, self.weights1)：將輸入數據與第一層權重相乘，進行矩陣乘法運算
        # 2. sigmoid()：將上述矩陣乘法的結果通過sigmoid激活函數，將值壓縮到0到1之間
        # 3. self.layer1：將sigmoid函數的結果存儲為隱藏層的輸出，供後續計算使用

        self.output = sigmoid(np.dot(self.layer1, self.weights2))  # 計算輸出層的輸出
        # 詳細解釋：
        # 1. np.dot(self.layer1, self.weights2)：將隱藏層的輸出與第二層權重相乘，進行矩陣乘法運算
        # 2. sigmoid()：將上述矩陣乘法的結果通過sigmoid激活函數，將值壓縮到0到1之間
        # 3. self.output：將sigmoid函數的結果存儲為神經網絡的最終輸出，這就是網絡的預測結果

    # 反向傳播函數
    def backprop(self):
        # 計算輸出層的誤差和權重調整
        d_weights2 = np.dot(self.layer1.T, (2*(self.y - self.output) * sigmoid_derivative(self.output)))
        # 詳細解釋：
        # 1. (self.y - self.output)：計算實際輸出與預期輸出之間的差異
        # 2. sigmoid_derivative(self.output)：計算輸出層的sigmoid函數導數
        # 3. 2*(self.y - self.output) * sigmoid_derivative(self.output)：計算輸出層的誤差項
        # 4. np.dot(self.layer1.T, ...)：將隱藏層的輸出與輸出層的誤差項進行矩陣乘法，得到第二層權重的調整量

        # 計算隱藏層的誤差和權重調整
        d_weights1 = np.dot(self.input.T,  (np.dot(2*(self.y - self.output) * sigmoid_derivative(self.output), self.weights2.T) * sigmoid_derivative(self.layer1)))
        # 詳細解釋：
        # 1. 2*(self.y - self.output) * sigmoid_derivative(self.output)：計算輸出層的誤差項
        # 2. np.dot(..., self.weights2.T)：將輸出層的誤差反向傳播到隱藏層
        # 3. sigmoid_derivative(self.layer1)：計算隱藏層的sigmoid函數導數
        # 4. ... * sigmoid_derivative(self.layer1)：計算隱藏層的誤差項
        # 5. np.dot(self.input.T, ...)：將輸入數據與隱藏層的誤差項進行矩陣乘法，得到第一層權重的調整量

        # 更新權重
        self.weights1 += d_weights1  # 更新第一層權重：將計算得到的調整量加到原權重上
        self.weights2 += d_weights2  # 更新第二層權重：將計算得到的調整量加到原權重上

    # 訓練網絡函數
    def train(self, iterations):
        for _ in range(iterations):  # 迭代指定的次數，進行多次訓練
            self.feedforward()  # 執行前向傳播，計算網絡輸出
            self.backprop()  # 執行反向傳播，更新網絡權重

# 使用示例
X = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1]])  # 定義輸入數據：4個樣本，每個樣本有3個特徵
y = np.array([[0], [1], [1], [0]])  # 定義目標輸出：對應的4個樣本的期望輸出

nn = NeuralNetwork(X, y)  # 創建神經網絡實例，使用給定的輸入數據和目標輸出初始化網絡
nn.train(1500)  # 訓練網絡 1500 次迭代，通過反覆的前向傳播和反向傳播來優化網絡權重

print(nn.output)  # 輸出最終結果：顯示經過訓練後網絡對輸入數據的預測結果

[[0.01379359]
 [0.97906213]
 [0.97993797]
 [0.02263262]]


## 損失計算：
1. 隱含的損失計算：
   雖然這個範例中沒有明確計算和輸出損失值，但損失的概念實際上是隱含在反向傳播過程中的。在 `backprop` 方法中，`(self.y - self.output)` 這部分實際上代表了預測值與真實值之間的差異，這就是損失的一種形式。

2. 簡化的實現：
   這個範例是一個簡化的神經網絡實現，主要目的是展示反向傳播的基本原理。為了保持代碼簡潔，省略了顯式的損失計算。


## 改進
為了使這個範例更完整，我們可以添加一個損失計算和輸出。這不僅能幫助我們監控訓練進度，還能判斷模型是否收斂。

以下是如何修改代碼以包含損失計算的建議：

```python:d:\Dev_prjs\AI_MIT\113-1_deep_mit\W06\sup02_back_progration_nerual_network.ipynb
class NeuralNetwork:
    # ... 其他代碼保持不變 ...

    def calculate_loss(self):
        # 使用均方誤差（MSE）作為損失函數
        return np.mean(np.square(self.y - self.output))

    def train(self, iterations):
        for i in range(iterations):
            self.feedforward()
            self.backprop()
            
            # 每100次迭代輸出一次損失
            if i % 100 == 0:
                loss = self.calculate_loss()
                print(f"迭代 {i}: 損失 = {loss}")

# 使用示例
X = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1]])
y = np.array([[0], [1], [1], [0]])

nn = NeuralNetwork(X, y)
nn.train(1500)

# 訓練結束後輸出最終損失
final_loss = nn.calculate_loss()
print(f"最終損失: {final_loss}")
print("最終輸出:")
print(nn.output)
```

這個修改版本添加了以下內容：
1. `calculate_loss` 方法用於計算均方誤差（MSE）。
2. 在 `train` 方法中，每100次迭代輸出一次當前的損失值。
3. 訓練結束後，輸出最終的損失值和網絡輸出。

通過這些修改，我們可以更好地追蹤訓練過程，觀察損失值的變化，從而判斷模型的訓練效果和收斂情況。這對於理解和改進神經網絡的訓練過程非常有幫助。

In [ ]:
import numpy as np  # 導入 NumPy 庫，用於進行數值計算和數組操作

# 定義 sigmoid 激活函數
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # 計算並返回 sigmoid 函數值

# 定義 sigmoid 函數的導數
def sigmoid_derivative(x):
    return x * (1 - x)  # 計算並返回 sigmoid 函數的導數

# 定義神經網絡類
class NeuralNetwork:
    def __init__(self, x, y):
        self.input = x  # 設置輸入數據
        self.weights1 = np.random.rand(self.input.shape[1], 4)  # 初始化第一層權重：使用numpy的random.rand函數生成一個隨機矩陣，大小為輸入特徵數量乘以4（隱藏層神經元數量）
        self.weights2 = np.random.rand(4, 1)  # 初始化第二層權重：生成一個4行1列的隨機矩陣，連接隱藏層到輸出層
        self.y = y  # 設置目標輸出：將傳入的目標值y賦給self.y，用於後續的訓練和誤差計算
        self.output = np.zeros(y.shape)  # 初始化輸出為零矩陣：創建一個與目標輸出y相同形狀的零矩陣，用於存儲網絡的預測結果

    # 前向傳播
    def feedforward(self):
        # 前向傳播函數
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))  # 計算隱藏層的輸出
        # 詳細解釋：
        # 1. np.dot(self.input, self.weights1)：將輸入數據與第一層權重相乘，進行矩陣乘法運算
        # 2. sigmoid()：將上述矩陣乘法的結果通過sigmoid激活函數，將值壓縮到0到1之間
        # 3. self.layer1：將sigmoid函數的結果存儲為隱藏層的輸出，供後續計算使用

        self.output = sigmoid(np.dot(self.layer1, self.weights2))  # 計算輸出層的輸出
        # 詳細解釋：
        # 1. np.dot(self.layer1, self.weights2)：將隱藏層的輸出與第二層權重相乘，進行矩陣乘法運算
        # 2. sigmoid()：將上述矩陣乘法的結果通過sigmoid激活函數，將值壓縮到0到1之間
        # 3. self.output：將sigmoid函數的結果存儲為神經網絡的最終輸出，這就是網絡的預測結果

    # 反向傳播函數
    def backprop(self):
        # 計算輸出層的誤差和權重調整
        d_weights2 = np.dot(self.layer1.T, (2*(self.y - self.output) * sigmoid_derivative(self.output)))
        # 詳細解釋：
        # 1. (self.y - self.output)：計算實際輸出與預期輸出之間的差異
        # 2. sigmoid_derivative(self.output)：計算輸出層的sigmoid函數導數
        # 3. 2*(self.y - self.output) * sigmoid_derivative(self.output)：計算輸出層的誤差項
        # 4. np.dot(self.layer1.T, ...)：將隱藏層的輸出與輸出層的誤差項進行矩陣乘法，得到第二層權重的調整量

        # 計算隱藏層的誤差和權重調整
        d_weights1 = np.dot(self.input.T,  (np.dot(2*(self.y - self.output) * sigmoid_derivative(self.output), self.weights2.T) * sigmoid_derivative(self.layer1)))
        # 詳細解釋：
        # 1. 2*(self.y - self.output) * sigmoid_derivative(self.output)：計算輸出層的誤差項
        # 2. np.dot(..., self.weights2.T)：將輸出層的誤差反向傳播到隱藏層
        # 3. sigmoid_derivative(self.layer1)：計算隱藏層的sigmoid函數導數
        # 4. ... * sigmoid_derivative(self.layer1)：計算隱藏層的誤差項
        # 5. np.dot(self.input.T, ...)：將輸入數據與隱藏層的誤差項進行矩陣乘法，得到第一層權重的調整量

        # 更新權重
        self.weights1 += d_weights1  # 更新第一層權重：將計算得到的調整量加到原權重上
        self.weights2 += d_weights2  # 更新第二層權重：將計算得到的調整量加到原權重上

    def calculate_loss(self):
        # 使用均方誤差（MSE）作為損失函數
        return np.mean(np.square(self.y - self.output))

    def train(self, iterations):
        for i in range(iterations):
            self.feedforward()
            self.backprop()
            
            # 每100次迭代輸出一次損失
            if i % 100 == 0:
                loss = self.calculate_loss()
                print(f"迭代 {i}: 損失 = {loss}")
                
# 使用示例
X = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1]])
y = np.array([[0], [1], [1], [0]])

nn = NeuralNetwork(X, y)
nn.train(1500)

# 訓練結束後輸出最終損失
final_loss = nn.calculate_loss()
print(f"最終損失: {final_loss}")
print("最終輸出:")
print(nn.output)